In [2]:
import datetime
import re
import cloudscraper
import pandas as pd 

from bs4 import BeautifulSoup

In [3]:
now = datetime.datetime.now()

In [4]:
root = "http://batdongsan.com.vn/nha-dat-ban-tp-hcm"
pages = set()

def getsubLinks(subUrl):
    '''Scrape root to get all sublinks by finding the last'''
    global pages
    url  = root + subUrl
    scraper = cloudscraper.create_scraper() #cloudscraper
    html = scraper.get(url).text
    soup = BeautifulSoup(html, "html.parser")

    last = max([int(link['pid']) for link in soup.findAll("a", href=re.compile(r"/p[0-9]"))])
    for i in range(2, int(last)+1): #starts at 2
        page = "/p{0}".format(str(i)) #ie. /p2, /p3, /p100
        if page not in pages:
            pages.add(page)

In [5]:
getsubLinks("")

In [25]:
# scraper = cloudscraper.create_scraper()  # returns a CloudScraper instance
# # # # Or: scraper = cloudscraper.CloudScraper()  # CloudScraper inherits from requests.Session
# html = scraper.get("https://batdongsan.com.vn/nha-dat-ban-tp-hcm/p2").text  # => "<!DOCTYPE html><html><head>..."
# soup = BeautifulSoup(html)
row={}
listing = soup.findAll("div", {"class":re.compile(".*product-item clearfix badReportItem")})[0]
prid      = listing['prid']
url       = "https://batdongsan.com.vn/nha-dat-ban-tp-hcm/p2"
title     = listing.find("a", {"class":"wrap-plink"})["title"] if listing.find("a", {"class":"wrap-plink"}) else None
price     = listing.find("span", {"class":"price"}).text if listing.find("span", {"class":"price"}) else None
area      = listing.find("span", {"class":"area"}).text if listing.find("span", {"class":"area"}) else None
bedroom   = listing.find("span", {"class":"bedroom"}).text if listing.find("span", {"class":"bedroom"}) else None
toilet    = listing.find("span", {"class":"toilet"}).text if listing.find("span", {"class":"toilet"}) else None
location  = listing.find("span", {"class":"location"}).text if listing.find("span", {"class":"location"}) else None
content   = listing.find("div", {"class":"product-content"}).text if listing.find("div", {"class":"product-content"}) else None
post_date = datetime.datetime.strptime(listing.find("span", {"class":"tooltip-time"}).text, "%d/%m/%Y").strftime("%Y-%m-%d") if listing.find("span", {"class":"tooltip-time"}) else None
price_vnd = convertPrice(listing.find("span", {"class":"price"}).text, listing.find("span", {"class":"area"}).text) if listing.find("span", {"class":"price"}) and listing.find("span", {"class":"area"}) else None
area_m2   = float(re.match("(.*?) m²", listing.find("span", {"class":"area"}).text).group(1)) if listing.find("span", {"class":"area"}) else None
city      = listing.find("span", {"class":"location"}).text.split(", ")[0] if listing.find("span", {"class":"location"}) else None
district  = listing.find("span", {"class":"location"}).text.split(", ")[1] if listing.find("span", {"class":"location"}) else None
scrape_timestamp = now
#https://stackoverflow.com/questions/10715965/create-pandas-dataframe-by-appending-one-row-at-a-time
row.update({
    "prid": prid,
    "price_vnd":price_vnd
})
print(row)


{'prid': '29364255', 'price_vnd': 56500000.0}


In [14]:
soup.findAll("div", {"class":re.compile(".*product-item clearfix badReportItem")})[0]['prid']

'29364255'

In [7]:
# def store(prid, url, title, price, area, bedroom, toilet, location, content, post_date, price_vnd, area_m2, city, district, scrape_timestamp):
#     try:
#         mySql_insert_query = """
#         INSERT INTO sales (prid, url, title, price, area, bedroom, toilet, location, content, post_date, price_vnd, area_m2, city, district, scrape_timestamp)
#         VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
#         """
#         record = (prid, url, title, price, area, bedroom, toilet, location, content, post_date, price_vnd, area_m2, city, district, scrape_timestamp)
#         cur.execute(mySql_insert_query, record)
#         cur.connection.commit()
        
#     except Exception as e:
#         print(e)

In [44]:
def convertPrice(price, area):
    '''Convert price format'''
    if price == "Giá thỏa thuận":
        return None
    elif price.split(" ")[1]=="triệu/m²":
            return float(re.match("(.*?) triệu/m²", price).group(1))*float(re.match("(.*?) m²", area).group(1))*1000000
    elif price.split(" ")[1]=="triệu":
        return float(re.match("(.*?) triệu", price).group(1))*1000000
    elif price.split(" ")[1]=="tỷ":
        return float(re.match("(.*?) tỷ", price).group(1))*10000000
    else:
        return price
    
def writeData(subUrls):
    '''Iterate through links and write rows''' 
    rows = []
    
    for subUrl in subUrls:
        print(subUrl)
        #grab html soup
        scraper = cloudscraper.create_scraper() #cloudscraper
        html = scraper.get(root+subUrl).text
        soup = BeautifulSoup(html, "html.parser")
        
        #listing per page
        for listing in soup.findAll("div", {"class":re.compile(".*product-item clearfix badReportItem")}):
            uid       = listing['uid']
            prid      = listing['prid']
            url       = root+subUrl
            title     = listing.find("a", {"class":"wrap-plink"})["title"] if listing.find("a", {"class":"wrap-plink"}) else None
            price     = listing.find("span", {"class":"price"}).text if listing.find("span", {"class":"price"}) else None
            area      = listing.find("span", {"class":"area"}).text if listing.find("span", {"class":"area"}) else None
            bedroom   = listing.find("span", {"class":"bedroom"}).text if listing.find("span", {"class":"bedroom"}) else None
            toilet    = listing.find("span", {"class":"toilet"}).text if listing.find("span", {"class":"toilet"}) else None
            location  = listing.find("span", {"class":"location"}).text if listing.find("span", {"class":"location"}) else None
            content   = listing.find("div", {"class":"product-content"}).text if listing.find("div", {"class":"product-content"}) else None
            post_date = datetime.datetime.strptime(listing.find("span", {"class":"tooltip-time"}).text, "%d/%m/%Y").strftime("%Y-%m-%d") if listing.find("span", {"class":"tooltip-time"}) else None
            price_vnd = convertPrice(listing.find("span", {"class":"price"}).text, listing.find("span", {"class":"area"}).text) if listing.find("span", {"class":"price"}) and listing.find("span", {"class":"area"}) else None
            area_m2   = float(re.match("(.*?) m²", listing.find("span", {"class":"area"}).text).group(1)) if listing.find("span", {"class":"area"}) else None
            city      = listing.find("span", {"class":"location"}).text.split(", ")[0] if listing.find("span", {"class":"location"}) else None
            district  = listing.find("span", {"class":"location"}).text.split(", ")[1] if listing.find("span", {"class":"location"}) else None
            scrape_timestamp = now
            
            row = {}
            row.update({
                "uid": uid,
                "prid": prid,
                "title": title,
                "price": price,
                "area": area,
                "bedroom": bedroom,
                "toilet": toilet,
                "location": location,
                "content": content,
                "post_date": post_date,
                "price_vnd": price_vnd,
                "area_m2": area_m2,
                "city": city,
                "district": district,
                "scrape_timestamp": scrape_timestamp
            })
            rows.append(row)
            
    return rows
#             store(prid, url, title, price, area, bedroom, toilet, location, content, post_date, price_vnd, area_m2, city, district, scrape_timestamp)

In [45]:
pages = set({'', '/p862'})
rows=writeData(pages)


/p862


In [46]:
df = pd.DataFrame(rows) 
df

,uid,prid,title,price,area,bedroom,toilet,location,content,post_date,price_vnd,area_m2,city,district,scrape_timestamp
0,236957,29387403,"Chương trình mở bán đặc biệt, biệt thự view sô...",Giá thỏa thuận,None,None,None,"Quận 9, Hồ Chí Minh",\n Nháº­n Booking c...,2021-04-08,NaN,NaN,Quận 9,Hồ Chí Minh,2021-04-08 18:16:25.617162
1,1445195,29387013,BÁN SUNRISE CITY - 2PN 106M2 - FULL NT - BAN ...,4 tỷ,106 m²,2 PN,2 WC,"Quận 7, Hồ Chí Minh",\n PhÃ²ng kinh doan...,2021-04-08,40000000.0,106.0,Quận 7,Hồ Chí Minh,2021-04-08 18:16:25.617162
2,897139,29385427,ONE PALACE 2 - NHÀ PHỐ COMPOUND VEN SÔNG QUẬN ...,4.8 tỷ,220 m²,5 PN,6 WC,"Quận 12, Hồ Chí Minh",\n Khu NhÃ phá» B...,2021-04-08,48000000.0,220.0,Quận 12,Hồ Chí Minh,2021-04-08 18:16:25.617162
3,1547887,29059844,"Độc quyền ""20 căn"" Hot nhất Feliz En Vista, gi...",4.5 tỷ,84 m²,2 PN,2 WC,"Quận 2, Hồ Chí Minh",\n Vá»i tiÃªu chÃ­...,2021-04-08,45000000.0,84.0,Quận 2,Hồ Chí Minh,2021-04-08 18:16:25.617162
4,1266546,26302316,Bán nhiều căn hộ Sunrise City.City View 1 3 4P...,3.4 tỷ,74 m²,None,None,"Quận 7, Hồ Chí Minh",\n ThÃ´ng tin mÃ´ t...,2021-04-08,34000000.0,74.0,Quận 7,Hồ Chí Minh,2021-04-08 18:16:25.617162
5,1281649,29380721,Bán nhà hẻm xe hơi đỗ trước nhà đường Đặng Dun...,17 tỷ,64 m²,None,None,"Quận 1, Hồ Chí Minh",\n BÃ¡n nhÃ háº»m ...,2021-04-08,170000000.0,64.0,Quận 1,Hồ Chí Minh,2021-04-08 18:16:25.617162
6,709815,28889943,"Chuyên căn hộ Eco Green SG, 1PN 2,6 tỷ; 2PN từ...",2.98 tỷ,52 m²,None,None,"Quận 7, Hồ Chí Minh",\n ThiÃªn 0909 288 ...,2021-04-08,29800000.0,52.0,Quận 7,Hồ Chí Minh,2021-04-08 18:16:25.617162
7,823641,29379598,Mở bán 19 nền đất từ chủ đầu tư. Liên kết trực...,15 triệu/m²,1 m²,None,None,"Bình Chánh, Hồ Chí Minh",\n 1) Lá»ch trÃ¬nh...,2021-04-08,15000000.0,1.0,Bình Chánh,Hồ Chí Minh,2021-04-08 18:16:25.617162
8,809664,29379489,Ngộp Covid bán Nhà xinh xắn mới xây tại hẻm 51...,5.5 tỷ,28.1 m²,None,None,"Phú Nhuận, Hồ Chí Minh",\n Ngá»p Covid bÃ¡...,2021-04-08,55000000.0,28.1,Phú Nhuận,Hồ Chí Minh,2021-04-08 18:16:25.617162
9,1522046,29367666,Nóng: Ngân hàng VIB hỗ trợ thanh lý 22 nền đất...,1.2 tỷ,80 m²,None,None,"Bình Chánh, Hồ Chí Minh",\n Dá»± Ã¡n ÄÃ£ cÃ...,2021-04-08,12000000.0,80.0,Bình Chánh,Hồ Chí Minh,2021-04-08 18:16:25.617162


In [30]:
# #begin connection
# conn = pymysql.connect(host='127.0.0.1', unix_socket='/tmp/mysql.sock',
#                        user='root', passwd='@Nddt050797', db='mysql', charset='utf8')
# cur = conn.cursor()
# cur.execute("USE scraping")

# random.seed(datetime.datetime.now())

# #get links
# getLinks("")
# pages.add('/nha-dat-ban') #add base

# #write data to DB
# try:
#     writeData(pages)
# finally:
#     cur.close()
#     conn.close()